In [273]:
#Import libraries
import numpy as np
import pandas as pd


In [274]:
#Loading data
file_path = 'data.npy'
loaded_array = np.load(file_path)

In [275]:
def polar_to_cartesian(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y


In [276]:
x_coord, y_coord = polar_to_cartesian(loaded_array[:,0], loaded_array[:,1])
scores = loaded_array[:,2]

In [277]:
def calc_gradients(x,y):
    x_non_zero = np.where(x == 0, 1, x)
    result = np.divide(y, x_non_zero)
    result = np.where(x == 0, np.nan, result)
    return result


In [278]:
slopes = calc_gradients(x_coord, y_coord)

In [279]:
intersection_y = lambda r,m,theta: r*(np.sin(theta) - m*np.cos(theta))

intersection_values_y = intersection_y(loaded_array[:,0],slopes,loaded_array[:,1])


In [280]:
intersection_x = lambda b,m: -b/m

intersection_values_x = intersection_x(intersection_values_y,slopes)

/var/folders/pn/7xb0xrx119l4_783pgg7gp140000gn/T/ipykernel_3478/1933300804.py:1: RuntimeWarning: invalid value encountered in divide
  intersection_x = lambda b,m: -b/m


In [281]:
#Getting indices of positive and negative oriented lines 

positive_gradient_lines = np.where(slopes[slopes >= 0])[0]
negative_gradient_lines = np.where(slopes[slopes < 0])[0]


In [282]:
#Create a new array that contains the x-intercept difference for each line of positive gradient to each line of negative gradient
x_intercept_diff = np.abs(intersection_values_x[positive_gradient_lines][:, np.newaxis] - intersection_values_x[negative_gradient_lines])
scores_diff = (scores[positive_gradient_lines][:, np.newaxis] + scores[negative_gradient_lines])

In [283]:

closest_pairs_data = []

for i,item in enumerate(x_intercept_diff):
    for j,element in enumerate(item):
        positive_grad = slopes[slopes >= 0][i]
        positive_intersection = intersection_values_y[i]
        negative_grad = slopes[slopes < 0][j]
        negative_intersection = intersection_values_y[j]
        x_intercept_difference = x_intercept_diff[i, j]
        sum_score = scores_diff[i, j]

        closest_pairs_data.append([
            positive_grad, negative_grad,positive_intersection,negative_intersection,
            x_intercept_difference, sum_score
        ])

column_names = ['Positive Gradient', 'Negative Gradient','Positive Y Intersection','Negative Y Intersection',
                'x-intercept Difference', 'Sum Score']

df = pd.DataFrame(closest_pairs_data, columns=column_names)
df = df.nsmallest(100, 'x-intercept Difference')

df


,Positive Gradient,Negative Gradient,Positive Y Intersection,Negative Y Intersection,x-intercept Difference,Sum Score
0,0.017455,-1.539865,0.000004,0.000004,0.0,136.0
468,0.052408,-1.482561,-0.000000,-0.000000,0.0,134.0
469,0.052408,-1.327044,-0.000000,-0.000000,0.0,130.0
470,0.052408,-1.539865,-0.000000,-0.000000,0.0,126.0
471,0.052408,-1.482561,-0.000000,-0.000000,0.0,126.0
...,...,...,...,...,...,...
617,0.052408,-11.430049,-0.000000,0.000000,0.0,97.0
618,0.052408,-8.144349,-0.000000,-0.000000,0.0,97.0
619,0.052408,-5.671281,-0.000000,-0.000000,0.0,97.0
620,0.052408,-5.144553,-0.000000,0.000000,0.0,97.0


In [284]:
df.to_csv('closest_pairs.csv', index=False)
